In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import seqpc

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Load data from facemocap_df.pkl
facemocap_df = pd.read_pickle('facemocap_df.pkl')
facemocap_df.sample(5)


,File name,Original SPC,Name,Experimentator,MoCap platform,Pathology,Repetitive,Movement,Date
239,P22_D1_M2x10e8.csv,"[[16.4007, 160.04, 644.917, 15.5284, 153.84, 6...",NaN,Mattieu,UTC,Facial palsy,False,2,NaN
102,P18_D2_M5x10e10.csv,"[[79.4738, 181.11, 634.844, 78.8641, 176.558, ...",NaN,Mattieu,UTC,Facial palsy,False,5,NaN
158,s05_s2_std.csv,"[[466.26, -73.942, 1174.85, 472.576, -63.7845,...",FRS02,François-Régis,UTC,False,False,,11-06-2014
88,JB01_P6.csv,"[[-47.9272, 533.19, 679.64, -51.3549, 549.191,...",JB01,François-Régis,UTC,False,False,,13-05-2014
239,CD01_M2bis.csv,"[[-336.822, 311.056, 1321.46, -321.945, 334.23...",CD01,François-Régis,UTC,False,False,2,12-05-2015


In [3]:
matrices = []
# Iterate over the rows of facemocap_df and store the values of 'Original SPC' in list_arrays if 'Pathology' is False, 'Repetitive' is False and 'Movement' is 1,2,3,4 or 5
for index, row in facemocap_df.iterrows():
    if (row['Pathology'] == False) & (row['Repetitive'] == False) & (row['Movement'] in [1,2,3,4,5]):
        original_spc = row['Original SPC']
        file_name = row['File name']
        spc, spc_array = seqpc.get_spc_from_df(facemocap_df, file_name, scaled=True, interpolated=True, dental_support_frame=False, target_length=100)
        matrices.append(spc_array[:, 3:-4, :].reshape(100, 303))


c:\Users\ederro\Documents\GitHub\test_3d\seqpc.py:36: RuntimeWarning: All-NaN slice encountered
  spc_array = spc_array - np.nanmin(np.nanmin(spc_array, axis=0), axis=0)
c:\Users\ederro\Documents\GitHub\test_3d\seqpc.py:37: RuntimeWarning: All-NaN slice encountered
  spc_array = spc_array / np.nanmax(np.nanmax(spc_array, axis=0), axis=0)


Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
Dental support was removed
D

In [4]:
imputer = SimpleImputer(strategy='mean')
imputed_matrices = [imputer.fit_transform(matrix) for matrix in matrices]


In [5]:
scaler = StandardScaler()
scaled_matrices = [scaler.fit_transform(matrix) for matrix in imputed_matrices]


In [6]:
data = np.concatenate(scaled_matrices, axis=0)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 294 and the array at index 5 has size 285

In [15]:
distance_matrix = nan_euclidean_distances(data)

In [25]:
eps = 0.01
min_samples = 5

In [26]:
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
clusters = dbscan.fit_predict(distance_matrix)


In [27]:
unique_clusters, counts = np.unique(clusters, return_counts=True)
print(f"Number of clusters: {len(unique_clusters)}")
print(f"Cluster sizes: {counts}")

Number of clusters: 1
Cluster sizes: [100]
